# Zomato Exercise

**Author/s:** Megan Sioco

**Course code and section:** DATA100 S17

**Student ID:** 12210382

**College:** CCS

Zomato is a restaurant aggregator website. The platform provides information, menus and user-reviews of restaurants as well as food delivery options from partner restaurants in select cities. You have been provided 3 datasets:

- zomato.csv contains restaurant information like average_cost_for_two, cuisines
- location.csv contains location of the restaurants like address, city, locality & etc
- ratings.csv contains aggregate rating for each restaurant

The different files are connected using the `id` columns which can be found in the different files. The id column is a unique identifier to represent a restaurant. You can use this to **merge** the different files.

In [9]:
import pandas as pd
import numpy as np

1. Merge the zomato, location & ratings file into a signle dataframe and store it to the `merged_df` variable **(3pts)**

In [11]:
df1 = pd.read_csv('location.csv')
df2 = pd.read_csv('ratings.csv')
df3 = pd.read_csv('zomato.csv')

df_temp_merge = pd.merge(df1, df2, on='id')
merged_df = pd.merge(df_temp_merge, df3, on='id')
merged_df.to_csv('merged_file.csv', index=False)

In [12]:
assert (6830, 35)==merged_df.shape

cols = ['id', 'address', 'city', 'city_id', 'country_id', 'latitude',
       'locality', 'locality_verbose', 'longitude', 'zipcode',
       'aggregate_rating', 'rating_color', 'rating_text', 'votes',
       'average_cost_for_two', 'book_url', 'cuisines', 'currency', 'deeplink',
       'events_url', 'featured_image', 'has_online_delivery',
       'has_table_booking', 'include_bogo_offers', 'is_book_form_web_view',
       'is_delivering_now', 'is_table_reservation_supported',
       'is_zomato_book_res', 'menu_url', 'mezzo_provider', 'name',
       'photos_url', 'price_range', 'thumb', 'url']

assert sorted(merged_df.columns) == sorted(cols)

2. Create a function that returns the average aggregate rating for each city. Expected returned value is a Series datatype which contains name of city as index and average aggregate rating as value. **(2pts)**

In [14]:
def exercise_2(merged_df):
    city_avg_rating = merged_df.groupby('city')['aggregate_rating'].mean()
    return city_avg_rating

In [40]:
res_series = exercise_2(merged_df)
assert True == np.isclose(3.302075, res_series[res_series.index == 'Pasay City'],
                          rtol=1e-05, atol=1e-08, equal_nan=False)[0]
assert True == np.isclose(1.890741, res_series[res_series.index == 'Valenzuela City'],
                          rtol=1e-05, atol=1e-08, equal_nan=False)[0]
assert True == np.isclose(1.558333, res_series[res_series.index == 'Malabon City'],
                          rtol=1e-05, atol=1e-08, equal_nan=False)[0]

3. Using the apply function of pandas convert the rating_text to a numeric equivalent. You are to introduce a new column in merged_df called `rating_numeric` which is based on the value from rating_text: **(3pts)**

Excellent = 5 <br>
Very Good = 4 <br>
Good = 3 <br>
Average = 2 <br>
Poor = 1 <br>
Not rated = 0 <br>

In [23]:
def exercise_3(merged_df):
    def rating_to_numeric(rating):
        mapping = {
            'Excellent': 5,
            'Very Good': 4,
            'Good': 3,
            'Average': 2,
            'Poor': 1,
            'Not rated': 0
        }
        return mapping.get(rating, 0)

    merged_df['rating_numeric'] = merged_df['rating_text'].apply(rating_to_numeric)
    return merged_df

In [38]:
assert 868 == merged_df[merged_df.rating_numeric == 0].shape[0]
assert True == ('rating_numeric' in merged_df.columns)

4. Create a function which filters the merged dataframe to only contain rows coming in from a specific city. Expected returned value is a dataframe **(2pts)**

In [34]:
def exercise_4(merged_df, city):
    filtered_df = merged_df[merged_df['city'] == city]
    return filtered_df

In [36]:
assert 'Makati City' == exercise_4(merged_df, 'Makati City').city.unique()[0]
assert (976, 36) == exercise_4(merged_df, 'Makati City').shape

assert 'Mandaluyong City' == exercise_4(merged_df, 'Mandaluyong City').city.unique()[0]
assert (443, 36) == exercise_4(merged_df, 'Mandaluyong City').shape

assert (0, 36) == exercise_4(merged_df, 'Quezon  City').shape

5. Create a function to filter the dataframe to only contain certain columns passed in the function. Expected returned value is a dataframe **(1pt)**

In [42]:
def exercise_5(merged_df, cols_to_filter):
    filtered_df = merged_df[cols_to_filter]
    return filtered_df

In [44]:
cols = ['name', 'cuisines', 'city', 'aggregate_rating']
assert sorted(cols) == sorted(exercise_5(merged_df, cols).columns)
assert (6830, 4) == exercise_5(merged_df, cols).shape

cols = ['average_cost_for_two', 'book_url', 'cuisines', 'currency', 'deeplink',
       'events_url', 'featured_image', 'has_online_delivery',
       'has_table_booking', 'include_bogo_offers', 'is_book_form_web_view']
assert sorted(cols) == sorted(exercise_5(merged_df, cols).columns)
assert (6830, 11) == exercise_5(merged_df, cols).shape

6.  What is the most common restaurant name in the dataset? **(2pts)** Write down your answer and show code used to come up with the answer.

In [69]:
def most_common(merged_df):
    num_restaurants = merged_df['name'].value_counts()
    common = num_restaurants.idxmax()
    num_common = num_restaurants.max()
    return common, num_common

common, num_common = most_common(merged_df)
print("The most common restaurant name, ", common, ", appeared ", num_common, " times in the dataset.", sep='')

The most common restaurant name, Starbucks, appeared 99 times in the dataset.


7. Which restaurant has the most expensive average cost for two? **(2pts)** Write down your answer and show code used to come up with the answer.

In [84]:
def most_expensive(merged_df):
    average_costs = merged_df.groupby('name')['average_cost_for_two'].mean()
    most_expensive = average_costs.idxmax()
    highest_cost = average_costs.max()
    return most_expensive, highest_cost

most_expensive, highest_cost = most_expensive(merged_df)
print(most_expensive, "has the most expensive cost for two at", highest_cost, "pesos.")

The Tasting Room has the most expensive cost for two at 10000.0 pesos.


8. How many restaurants do not have a zipcode? **(1pt)** Write down your answer and show code used to come up with the answer.

In [72]:
def no_zipcode(merged_df):
    count = merged_df['zipcode'].isna().sum()
    return count

count = no_zipcode(merged_df)
print("The number of restaurants without a zipcode is", count)

The number of restaurants without a zipcode is 6713


9. How many restaurants in the dataset allow online delivery? **(1pt)** Write down your answer and show code used to come up with the answer.

In [75]:
def online_delivery(merged_df):
    count = merged_df['has_online_delivery'].sum()
    return count

count = online_delivery(merged_df)
print("The number of restaurants with online delivery is", count)

The number of restaurants with online delivery is 0


10. Which locatlity has more expensive food? Poblacion or Salcedo Village? **(3pts)** Write down your answer and show code used to come up with the answer.


In [92]:
def average_cost(merged_df, locality):
    average_cost = merged_df[merged_df['locality'] == locality]['average_cost_for_two'].mean()
    return average_cost

cost_poblacion = average_cost(merged_df, 'Poblacion')
cost_salcedo = average_cost(merged_df, 'Salcedo Village')

if cost_poblacion > cost_salcedo:
    print("Poblacion is more expensive, costing", cost_poblacion, "pesos.")
else:
    print("Salcedo Village is more expensive, costing", cost_salcedo, "pesos.")

Salcedo Village is more expensive, costing 989.1025641025641 pesos.
